![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

#🔎 Financial Text Splitting

Text splitting is very important for many reasons.

- Financial documents may be **very long** (e.g., filings, annual/quarterly reports) and applying AI to the whole document may take much time.
- Most Language Models have **token restrictions**. For example, Bert-based models can only process up to 512 tokens, while our biggest LM (Longformers) can process up to 4096 (but affecting negatively to the performance).
- If you have models which do specific extractions on **specific sections**, it does not make sense you run them over the whole document: you may get a lot of False Positives.

But also:
- Language Models will only "understand" only that **portion of text you send to them**. If you send *little information*, they may be biased and take wrong decisions.
- On the other hand, sometimes sending *too much information* makes the information get deluded or add noise.

And!
- Make sure the tokenization and splitting mechanism is the same in training and inference, or you will get undesirable results.

Because of these reasons, the very first thing you need to think about is how to properly split your texts, depending on the kind of task you want to carry out.

In [ ]:
from johnsnowlabs import *


#✔ 1. Text Classification

#✔ 1.1. Background
Text classification is the NLP task in charge of retrieving a `category` based on a piece of text you send to the model. A good example of it it's Section or Item identification in Financial Documents.

📚There are several ways we can carry out Text Classification:

- `At a whole-text level (no splitting)`: That's not feasible for most financial documents. As we already know, we have a token restrictions (512 for most BERT-based thansformers). We can use Longformers (4096), but in most cases, documents as Financial Reports have much more than that, what will go for sure beyond any limitation.

- `Retrieving first page`: In most cases, the relevant information of a document is in the first page. Just by splitting by pages and retrieving the first one, you can do text classification. 

- `At paragraph, section or sentence level using Finance NLP TextSplitter`: `TextSplitter` is an NLP annotator that it's meant to split documents into sentences, but is pretty customisable to retrieve paragraphs or setions as well. It works with `regex` or list of character splitting, among other more complex techniques.

- `At paragraph, section or sentence level using NER` for detecting headers and `ChunkSentenceSplitter`

The way you split may totally change the results you are getting. Let's see an example.

#✔ 1.2. Page splitting

Sometimes, pages have patterns which tell you how to split them. In our case, `Table of Contents` was present in the bottom of our documents.

Feel free to always analyze for signals when trying to detect pages boundaries. Patterns you can usually find in the bottom of a page:
- Number of page
- Bottom placeholders
- Name of people
- Name of the document
- etc.

In [ ]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

text_splitter = finance.TextSplitter() \
    .setInputCols(["document"]) \
    .setOutputCol("pages")\
    .setCustomBounds(["Table of Contents"])\
    .setUseCustomBoundsOnly(True)\
    .setExplodeSentences(True)

nlp_pipeline = nlp.Pipeline(stages=[
    document_assembler,
    text_splitter])

📜Explanation:
- `.setCustomBounds(["Table of Contents"])` sets an array of regular expression(s) to tell the annotator how to split the document.
- `.setUseCustomBoundsOnly(True)` the default behaviour of TextSplitter is Text Splitting, so we set to ignore the default regex ('\n', ...).
- `.setExplodeSentences(True)` creates one new row in the dataframe per split.

Let's download a document and use `whole-text` classifiers.

In [ ]:
import requests
URL = "https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/finance-nlp/data/cdns-20220101.html.txt"
response = requests.get(URL)

cadence_sec10k = response.content.decode('utf-8')

Let's apply our page splitter.

## Using Fit/Transform (all the dataframe will be processed in paralel using all the spark cluster nodes)

In [ ]:
#fit: trains, configures and prepares the pipeline for inference. 

sdf = spark.createDataFrame([[ cadence_sec10k ]]).toDF("text")

fit = nlp_pipeline.fit(sdf)

In [ ]:
%%time

#transforms: executes inference on a fit pipeline
res = fit.transform(sdf)

res.show()

+--------------------+--------------------+--------------------+
 text| document| pages|
+--------------------+--------------------+--------------------+
Table of Contents...|[{document, 0, 34...|[{document, 18, 4...|
Table of Contents...|[{document, 0, 34...|[{document, 4087,...|
Table of Contents...|[{document, 0, 34...|[{document, 4215,...|
Table of Contents...|[{document, 0, 34...|[{document, 5504,...|
Table of Contents...|[{document, 0, 34...|[{document, 11617...|
Table of Contents...|[{document, 0, 34...|[{document, 13985...|
Table of Contents...|[{document, 0, 34...|[{document, 20001...|
Table of Contents...|[{document, 0, 34...|[{document, 26059...|
Table of Contents...|[{document, 0, 34...|[{document, 31638...|
Table of Contents...|[{document, 0, 34...|[{document, 36733...|
Table of Contents...|[{document, 0, 34...|[{document, 42440...|
Table of Contents...|[{document, 0, 34...|[{document, 47053...|
Table of Contents...|[{document, 0, 34...|[{document, 48328...|
Table of Contents...|[{document, 0, 34...|[{document, 53745...|
Table of Contents...|[{document, 0, 34...|[{document, 59341...|
Table of Contents...|[{document, 0, 34...|[{document, 65403...|
Table of Contents...|[{document, 0, 34...|[{document, 72330...|
Table of Contents...|[{document, 0, 34...|[{document, 77951...|
Table of Contents...|[{document, 0, 34...|[{document, 84131...|
Table of Contents...|[{document, 0, 34...|[{document, 89718...|
+--------------------+--------------------+--------------------+
only showing top 20 rows

CPU times: user 16.9 ms, sys: 2.97 ms, total: 19.9 ms
Wall time: 2.78 s

## Using LightPipelines (everything will be executed in 1 node. It's much quicker for small dataframes, but does not leverage the cluster capacities)

In [ ]:
%%time
import json
lp = nlp.LightPipeline(fit)

json_res = lp.annotate(cadence_sec10k)

print(json.dumps(json_res, indent=4))

{
 "document": [
 "Table of Contents\nUNITED STATES SECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n_____________________________________ \nFORM 10-K \n_____________________________________ \n(Mark One)\n\u2612\nANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended January 1, 2022 \nOR\n\u2610\nTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from _________ to_________.\n\nCommission file number 000-15867 \n_____________________________________\n \nCADENCE DESIGN SYSTEMS, INC. \n(Exact name of registrant as specified in its charter)\n____________________________________ \nDelaware\n \n00-0000000\n(State or Other Jurisdiction ofIncorporation or Organization)\n \n(I.R.S. EmployerIdentification No.)\n2655 Seely Avenue, Building 5,\nSan Jose,\nCalifornia\n \n95134\n(Address of Principal Executive Offices)\n \n(Zip Code)\n(408)\n-943-1234 \n(Registrant\u2019s Telephone Number, including Area Code) \nSecurities registered pursuant to Section 12(b) of the Act:\nTitle of Each Class\nTrading Symbol(s)\nNames of Each Exchange on which Registered\nCommon Stock, $0.01 par value per share\nCDNS\nNasdaq Global Select Market\nSecurities registered pursuant to Section 12(g) of the Act:\nNone\nIndicate by check mark if the registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act. \n Yes \n\u2612\n No \n\u2610\nIndicate by check mark if the registrant is not required to file reports pursuant to Section 13 or Section 15(d) of the Act. \n Yes \n\u2610 \nNo \n\u2612\nIndicate by check mark whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange Act of 1934 during the preceding 12 months (or for such shorter period that the registrant was required to file such reports), and (2) has been subject to such filing requirements for the past 90 days. \n Yes \n\u2612\n No \n\u2610\nIndicate by check mark whether the registrant has submitted electronically every Interactive Data File required to be submitted pursuant to Rule 405 of Regulation S-T (\u00a7 232.405 of this chapter) during the preceding 12 months (or for such shorter period that the registrant was required to submit such files). \n Yes \n\u2612\n No \n\u2610\nIndicate by check mark whether the registrant is a large accelerated filer, an accelerated filer, a non-accelerated filer, a smaller reporting company, or an emerging growth company. See the definitions of \u201clarge accelerated filer,\u201d \u201caccelerated filer,\u201d \u201csmaller reporting company,\u201d and \u201cemerging growth company\u201d in Rule 12b-2 of the Exchange Act.\nLarge Accelerated Filer\n\u2612\nAccelerated Filer\n\u2610\nNon-accelerated Filer\n\u2610\nSmaller Reporting Company\n\u2610\nEmerging Growth Company\n\u2610\nIf an emerging growth company, indicate by check mark if the registrant has elected not to use the extended transition period for complying with any new or revised financial accounting standards provided pursuant to Section 13(a) of the Exchange Act. \n\u2610\nIndicate by check mark whether the registrant has filed a report on and attestation to its management\u2019s assessment of the effectiveness of its internal control over financial reporting under Section 404(b) of the Sarbanes-Oxley Act (15 U.S.C. 7262(b)) by the registered public accounting firm that prepared or issued its audit report. \n\u2612\nIndicate by check mark whether the registrant is a shell company (as defined in Rule 12b-2 of the Act). \n Yes \n\u2610 \nNo \n\u2612\nThe aggregate market value of the voting and non-voting common equity held by non-affiliates computed by reference to the price at which the common equity was last sold as of the last business day of the registrant\u2019s most recently completed second fiscal quarter ended July 3, 2021 was approximately $38,179,000,000.\nOn February 5, 2022, 

In [ ]:
pages = [json_res['pages'][i] for i in range(0,20)]

#✔ 1.3. Document Classification using the 1st page
Most of the documents can be identified by the 1 page (given it's a real 1st page and not a separation, cover page, page with noise, etc - you can filter those out).

![image.png](/files/FINLEG/dc1.png)

📚We have some text classifiers which can be used to identify different financial reports:
- `finclf_sec_filings_en`: This model allows you to classify documents among a list of specific US Security Exchange Commission filings, as `10-K, 10-Q, 8-K, S-8, 3, 4, Other`. IMPORTANT : This model works with the first 512 tokens of a document, you don't need to run it in the whole document.`
- `finclf_earning_broker_10k_en`: This is a Text Cassification model, which can help you identify if a model is an Earning Call, a Broker Report, a 10K filing or something else. IMPORTANT : This model works with the first 512 tokens of a document, you don't need to run it in the whole document.`
- ...
- missing something? Let us know and we will train that for you!

In [ ]:
documentAssembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

embeddings = nlp.BertSentenceEmbeddings.pretrained("sent_bert_base_cased", "en") \
    .setInputCols("document") \
    .setOutputCol("sentence_embeddings")

docClassifier = finance.ClassifierDLModel.pretrained("finclf_earning_broker_10k", "en", "finance/models")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("label") \

nlp_pipeline = nlp.Pipeline(stages=[
    documentAssembler,
    embeddings,
    docClassifier])

sent_bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[ | ][OK!]
finclf_earning_broker_10k download started this may take some time.
[ | ][OK!]

### Using fit / transform (all Spark Cluster nodes)

In [ ]:
# Since the model works with the first 512 tokens, let's just send some initial characters...
sdf = spark.createDataFrame([[ pages[0] ]]).toDF("text")

fit = nlp_pipeline.fit(sdf)
res = fit.transform(sdf)

res = res.select('label.result').show()

+------+
result|
+------+
 [10k]|
+------+

### Using Light Pipeline (only driver, quicker but not scalable for big amounts of documents)

In [ ]:
from johnsnowlabs import nlp
import json

lp = nlp.LightPipeline(fit)
json_res = lp.annotate( pages[0] )
print(json.dumps(json_res, indent=4))

{
 "document": [
 "UNITED STATES SECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n_____________________________________ \nFORM 10-K \n_____________________________________ \n(Mark One)\n\u2612\nANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended January 1, 2022 \nOR\n\u2610\nTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from _________ to_________.\n\nCommission file number 000-15867 \n_____________________________________\n \nCADENCE DESIGN SYSTEMS, INC. \n(Exact name of registrant as specified in its charter)\n____________________________________ \nDelaware\n \n00-0000000\n(State or Other Jurisdiction ofIncorporation or Organization)\n \n(I.R.S. EmployerIdentification No.)\n2655 Seely Avenue, Building 5,\nSan Jose,\nCalifornia\n \n95134\n(Address of Principal Executive Offices)\n \n(Zip Code)\n(408)\n-943-1234 \n(Registrant\u2019s Telephone Number, including Area Code) \nSecurities registered pursuant to Section 12(b) of the Act:\nTitle of Each Class\nTrading Symbol(s)\nNames of Each Exchange on which Registered\nCommon Stock, $0.01 par value per share\nCDNS\nNasdaq Global Select Market\nSecurities registered pursuant to Section 12(g) of the Act:\nNone\nIndicate by check mark if the registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act. \n Yes \n\u2612\n No \n\u2610\nIndicate by check mark if the registrant is not required to file reports pursuant to Section 13 or Section 15(d) of the Act. \n Yes \n\u2610 \nNo \n\u2612\nIndicate by check mark whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange Act of 1934 during the preceding 12 months (or for such shorter period that the registrant was required to file such reports), and (2) has been subject to such filing requirements for the past 90 days. \n Yes \n\u2612\n No \n\u2610\nIndicate by check mark whether the registrant has submitted electronically every Interactive Data File required to be submitted pursuant to Rule 405 of Regulation S-T (\u00a7 232.405 of this chapter) during the preceding 12 months (or for such shorter period that the registrant was required to submit such files). \n Yes \n\u2612\n No \n\u2610\nIndicate by check mark whether the registrant is a large accelerated filer, an accelerated filer, a non-accelerated filer, a smaller reporting company, or an emerging growth company. See the definitions of \u201clarge accelerated filer,\u201d \u201caccelerated filer,\u201d \u201csmaller reporting company,\u201d and \u201cemerging growth company\u201d in Rule 12b-2 of the Exchange Act.\nLarge Accelerated Filer\n\u2612\nAccelerated Filer\n\u2610\nNon-accelerated Filer\n\u2610\nSmaller Reporting Company\n\u2610\nEmerging Growth Company\n\u2610\nIf an emerging growth company, indicate by check mark if the registrant has elected not to use the extended transition period for complying with any new or revised financial accounting standards provided pursuant to Section 13(a) of the Exchange Act. \n\u2610\nIndicate by check mark whether the registrant has filed a report on and attestation to its management\u2019s assessment of the effectiveness of its internal control over financial reporting under Section 404(b) of the Sarbanes-Oxley Act (15 U.S.C. 7262(b)) by the registered public accounting firm that prepared or issued its audit report. \n\u2612\nIndicate by check mark whether the registrant is a shell company (as defined in Rule 12b-2 of the Act). \n Yes \n\u2610 \nNo \n\u2612\nThe aggregate market value of the voting and non-voting common equity held by non-affiliates computed by reference to the price at which the common equity was last sold as of the last business day of the registrant\u2019s most recently completed second fiscal quarter ended July 3, 2021 was approximately $38,179,000,000.\nOn February 5, 2022, approximately 277,3

### Are you curious of what would happen with other pages❓

In [ ]:
json_res = lp.annotate( [ pages[0], pages[1], pages[5], pages[10], pages[15], pages[19]] )
print(json.dumps(json_res, indent=4))

[
 {
 "document": [
 "UNITED STATES SECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n_____________________________________ \nFORM 10-K \n_____________________________________ \n(Mark One)\n\u2612\nANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended January 1, 2022 \nOR\n\u2610\nTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from _________ to_________.\n\nCommission file number 000-15867 \n_____________________________________\n \nCADENCE DESIGN SYSTEMS, INC. \n(Exact name of registrant as specified in its charter)\n____________________________________ \nDelaware\n \n00-0000000\n(State or Other Jurisdiction ofIncorporation or Organization)\n \n(I.R.S. EmployerIdentification No.)\n2655 Seely Avenue, Building 5,\nSan Jose,\nCalifornia\n \n95134\n(Address of Principal Executive Offices)\n \n(Zip Code)\n(408)\n-943-1234 \n(Registrant\u2019s Telephone Number, including Area Code) \nSecurities registered pursuant to Section 12(b) of the Act:\nTitle of Each Class\nTrading Symbol(s)\nNames of Each Exchange on which Registered\nCommon Stock, $0.01 par value per share\nCDNS\nNasdaq Global Select Market\nSecurities registered pursuant to Section 12(g) of the Act:\nNone\nIndicate by check mark if the registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act. \n Yes \n\u2612\n No \n\u2610\nIndicate by check mark if the registrant is not required to file reports pursuant to Section 13 or Section 15(d) of the Act. \n Yes \n\u2610 \nNo \n\u2612\nIndicate by check mark whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange Act of 1934 during the preceding 12 months (or for such shorter period that the registrant was required to file such reports), and (2) has been subject to such filing requirements for the past 90 days. \n Yes \n\u2612\n No \n\u2610\nIndicate by check mark whether the registrant has submitted electronically every Interactive Data File required to be submitted pursuant to Rule 405 of Regulation S-T (\u00a7 232.405 of this chapter) during the preceding 12 months (or for such shorter period that the registrant was required to submit such files). \n Yes \n\u2612\n No \n\u2610\nIndicate by check mark whether the registrant is a large accelerated filer, an accelerated filer, a non-accelerated filer, a smaller reporting company, or an emerging growth company. See the definitions of \u201clarge accelerated filer,\u201d \u201caccelerated filer,\u201d \u201csmaller reporting company,\u201d and \u201cemerging growth company\u201d in Rule 12b-2 of the Exchange Act.\nLarge Accelerated Filer\n\u2612\nAccelerated Filer\n\u2610\nNon-accelerated Filer\n\u2610\nSmaller Reporting Company\n\u2610\nEmerging Growth Company\n\u2610\nIf an emerging growth company, indicate by check mark if the registrant has elected not to use the extended transition period for complying with any new or revised financial accounting standards provided pursuant to Section 13(a) of the Exchange Act. \n\u2610\nIndicate by check mark whether the registrant has filed a report on and attestation to its management\u2019s assessment of the effectiveness of its internal control over financial reporting under Section 404(b) of the Sarbanes-Oxley Act (15 U.S.C. 7262(b)) by the registered public accounting firm that prepared or issued its audit report. \n\u2612\nIndicate by check mark whether the registrant is a shell company (as defined in Rule 12b-2 of the Act). \n Yes \n\u2610 \nNo \n\u2612\nThe aggregate market value of the voting and non-voting common equity held by non-affiliates computed by reference to the price at which the common equity was last sold as of the last business day of the registrant\u2019s most recently completed second fiscal quarter ended July 3, 2021 was approximately $38,179,000,000.\nOn February 5, 2022, approximately 27

Ok our classifiers, which work on the top 512 tokens, say we have a **10K document** by using the first page.

#✔ 1.4. Paragraph / Section / Items splitting

##📌 TextSplitter
To split by sentences or paragraphs or ...

In most cases:
- paragraphs can be extracted using just `\n\n`.
- sections and items may be delimited by headers and subheaders. We can train DL models to detect them or, if use numeration (if they are numerated sections) or other patterns to delimit their boundaries

In [ ]:
example1 = """PART I

ITEM 1. BUSINESS.
Company Overview
Inuvo is a technology company that develops and sells information technology solutions for marketing.

ITEM 1A. RISK FACTORS.
An investment in our common stock involves a significant degree of risk.

ITEM 1B. UNRESOLVED STAFF COMMENTS.
Not applicable to a smaller reporting company.

ITEM 2. PROPERTIES.
Our corporate headquarters are located in Little Rock, Arkansas where we entered into a five-year agreement to lease office space on October 1, 2015 and amended the lease as of February 1, 2021
Angeles, CA, San Jose, CA and Secaucus, NJ.

ITEM 3. LEGAL PROCEEDINGS.
We are not party to any pending legal proceedings.

ITEM 4. MINE SAFETY DISCLOSURES.
Not applicable.

PART II
ITEM 5. MARKET FOR REGISTRANT’S COMMON EQUITY, RELATED STOCKHOLDER MATTERS AND ISSUER PURCHASES OF EQUITY SECURITIES.
Market Information"""

By `\n\n`...

In [ ]:
text_splitter = finance.TextSplitter() \
    .setInputCols(["document"]) \
    .setOutputCol("paragraphs")\
    .setCustomBounds(["\n\n"])\
    .setUseCustomBoundsOnly(True)\
    .setExplodeSentences(True)

nlp_pipeline = nlp.Pipeline(stages=[
    document_assembler,
    text_splitter])

sdf = spark.createDataFrame([[ example1 ]]).toDF("text")

fit = nlp_pipeline.fit(sdf)
res = fit.transform(sdf)

res = res.select('paragraphs.result').show(truncate = False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
result |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[PART I] |
[ITEM 1. BUSINESS.\nCompany Overview\nInuvo is a technology company that develops and sells information technology solutions for marketing.] |
[ITEM 1A. RISK FACTORS.\nAn investment in our common stock involves a significant degree of risk.] |
[ITEM 1B. UNRESOLVED STAFF COMMENTS.\nNot applicable to a smaller reporting company.] |
[ITEM 2. PROPERTIES.\nOur corporate headquarters are located in Little Rock, Arkansas where we entered into a five-year agreement to lease office space on October 1, 2015 and amended the lease as of February 1, 2021\nAngeles, CA, San Jose, CA and Secaucus, NJ.]|
[ITEM 3. LEGAL PROCEEDINGS.\nWe are not party to any pending legal proceedings.] |
[ITEM 4. MINE SAFETY DISCLOSURES.\nNot applicable.] |
[PART II\nITEM 5. MARKET FOR REGISTRANT’S COMMON EQUITY, RELATED STOCKHOLDER MATTERS AND ISSUER PURCHASES OF EQUITY SECURITIES.\nMarket Information] |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

In [ ]:
text_splitter = finance.TextSplitter() \
    .setInputCols(["document"]) \
    .setOutputCol("paragraphs")\
    .setCustomBounds(["\n\n"])\
    .setUseCustomBoundsOnly(True)\
    .setExplodeSentences(True)

nlp_pipeline = nlp.Pipeline(stages=[
    document_assembler,
    text_splitter])

sdf = spark.createDataFrame([[ example1 ]]).toDF("text")

fit = nlp_pipeline.fit(sdf)
res = fit.transform(sdf)

res = res.select('paragraphs.result').show(truncate = False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
result |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[PART I] |
[ITEM 1. BUSINESS.\nCompany Overview\nInuvo is a technology company that develops and sells information technology solutions for marketing.] |
[ITEM 1A. RISK FACTORS.\nAn investment in our common stock involves a significant degree of risk.] |
[ITEM 1B. UNRESOLVED STAFF COMMENTS.\nNot applicable to a smaller reporting company.] |
[ITEM 2. PROPERTIES.\nOur corporate headquarters are located in Little Rock, Arkansas where we entered into a five-year agreement to lease office space on October 1, 2015 and amended the lease as of February 1, 2021\nAngeles, CA, San Jose, CA and Secaucus, NJ.]|
[ITEM 3. LEGAL PROCEEDINGS.\nWe are not party to any pending legal proceedings.] |
[ITEM 4. MINE SAFETY DISCLOSURES.\nNot applicable.] |
[PART II\nITEM 5. MARKET FOR REGISTRANT’S COMMON EQUITY, RELATED STOCKHOLDER MATTERS AND ISSUER PURCHASES OF EQUITY SECURITIES.\nMarket Information] |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

By other regular expressions ...

In [ ]:
example2 = """PART I
ITEM 1. BUSINESS.
Company Overview
Inuvo is a technology company that develops and sells information technology solutions for marketing.
ITEM 1A. RISK FACTORS.
An investment in our common stock involves a significant degree of risk.
ITEM 1B. UNRESOLVED STAFF COMMENTS.
Not applicable to a smaller reporting company.
ITEM 2. PROPERTIES.
Our corporate headquarters are located in Little Rock, Arkansas where we entered into a five-year agreement to lease office space on October 1, 2015 and amended the lease as of February 1, 2021
Angeles, CA, San Jose, CA and Secaucus, NJ.
ITEM 3. LEGAL PROCEEDINGS.
We are not party to any pending legal proceedings.
ITEM 4. MINE SAFETY DISCLOSURES.
Not applicable.
PART II
ITEM 5. MARKET FOR REGISTRANT’S COMMON EQUITY, RELATED STOCKHOLDER MATTERS AND ISSUER PURCHASES OF EQUITY SECURITIES.
Market Information"""

In [ ]:
text_splitter = finance.TextSplitter() \
    .setInputCols(["document"]) \
    .setOutputCol("paragraphs")\
    .setCustomBounds(["[A-Z]{4,}[ ]*[0-9]*[A-Z]*\.? ?"])\
    .setUseCustomBoundsOnly(True)\
    .setExplodeSentences(True)

nlp_pipeline = nlp.Pipeline(stages=[
    document_assembler,
    text_splitter])

sdf = spark.createDataFrame([[ example2 ]]).toDF("text")

fit = nlp_pipeline.fit(sdf)
res = fit.transform(sdf)

res = res.select('paragraphs.result').show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
result |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[Company Overview\nInuvo is a technology company that develops and sells information technology solutions for marketing.] |
[An investment in our common stock involves a significant degree of risk.] |
[Not applicable to a smaller reporting company.] |
[Our corporate headquarters are located in Little Rock, Arkansas where we entered into a five-year agreement to lease office space on October 1, 2015 and amended the lease as of February 1, 2021\nAngeles, CA, San Jose, CA and Secaucus, NJ.]|
[We are not party to any pending legal proceedings.] |
[Not applicable.] |
[’S] |
[,] |
[OF] |
[Market Information] |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

BE CAREFUL. If you use a pattern to split and you want it to be included, use `setCustomBoundsStrategy` to `prepend` (goes as the first part of the next section) or `append` (goes as a last part of the previous section)

In [ ]:
text_splitter = finance.TextSplitter() \
    .setInputCols(["document"]) \
    .setOutputCol("paragraphs")\
    .setCustomBounds(["[A-Z]{4,}[ ]*[0-9]*[A-Z]*\.? ?"])\
    .setUseCustomBoundsOnly(True)\
    .setExplodeSentences(True)\
    .setCustomBoundsStrategy('prepend')

nlp_pipeline = nlp.Pipeline(stages=[
    document_assembler,
    text_splitter])

sdf = spark.createDataFrame([[ example2 ]]).toDF("text")

fit = nlp_pipeline.fit(sdf)
res = fit.transform(sdf)


res = res.select('paragraphs.result').show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
result |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[PART I] |
[ITEM 1.] |
[BUSINESS.\nCompany Overview\nInuvo is a technology company that develops and sells information technology solutions for marketing.] |
[ITEM 1A.] |
[RISK FACTORS.\nAn investment in our common stock involves a significant degree of risk.] |
[ITEM 1B.] |
[UNRESOLVED STAFF] |
[COMMENTS.\nNot applicable to a smaller reporting company.] |
[ITEM 2.] |
[PROPERTIES.\nOur corporate headquarters are located in Little Rock, Arkansas where we entered into a five-year agreement to lease office space on October 1, 2015 and amended the lease as of February 1, 2021\nAngeles, CA, San Jose, CA and Secaucus, NJ.]|
[ITEM 3.] |
[LEGAL PROCEEDINGS.\nWe are not party to any pending legal proceedings.] |
[ITEM 4.] |
[MINE SAFETY] |
[DISCLOSURES.\nNot applicable.] |
[PART II] |
[ITEM 5.] |
[MARKET FOR] |
[REGISTRANT’S] |
[COMMON EQUITY,] |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 20 rows

##📌 Using NER and ChunkSentenceSplitter

... using pretrained NER models for `Headers and Subheaders`

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

sentenceDetector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl","xx")\
        .setInputCols(["document"])\
        .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
        .setInputCols(["sentence"])\
        .setOutputCol("token")

embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en") \
        .setInputCols(["sentence", "token"]) \
        .setOutputCol("embeddings")

ner_model = finance.NerModel.pretrained("finner_headers", "en", "finance/models")\
        .setInputCols(["sentence", "token", "embeddings"])\
        .setOutputCol("ner")

ner_converter = finance.NerConverterInternal()\
        .setInputCols(["sentence","token","ner"])\
        .setOutputCol("ner_chunk")

nlp_pipeline = nlp.Pipeline(stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        embeddings,
        ner_model,
        ner_converter])

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[ | ][OK!]
bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][OK!]
finner_headers download started this may take some time.
[ | ][OK!]

In [ ]:
sdf = spark.createDataFrame([[ example2 ]]).toDF("text")

fit = nlp_pipeline.fit(sdf)

lp = nlp.LightPipeline(fit)

In [ ]:
res = lp.fullAnnotate(example2)

In [ ]:
from johnsnowlabs import viz

ner_viz = viz.NerVisualizer()

displayHTML(ner_viz.display(res[0], label_col='ner_chunk', return_html = True))

PART I HEADER ITEM 1. BUSINESS HEADER . Company Overview SUBHEADER Inuvo is a technology company that develops and sells information technology solutions for marketing. ITEM 1A SUBHEADER . RISK FACTORS SUBHEADER . An investment in our common stock involves a significant degree of risk. ITEM 1B. UNRESOLVED STAFF COMMENTS SUBHEADER . Not applicable to a smaller reporting company. ITEM 2 HEADER . PROPERTIES SUBHEADER . Our corporate headquarters are located in Little Rock, Arkansas where we entered into a five-year agreement to lease office space on October 1, 2015 and amended the lease as of February 1, 2021 Angeles, CA, San Jose, CA and Secaucus, NJ. ITEM 3. HEADER LEGAL PROCEEDINGS SUBHEADER . We are not party to any pending legal proceedings. ITEM 4. MINE SAFETY DISCLOSURES HEADER . Not applicable SUBHEADER . PART II HEADER ITEM 5. MARKET FOR REGISTRANT’S COMMON EQUITY, RELATED STOCKHOLDER MATTERS AND ISSUER PURCHASES OF EQUITY SECURITIES HEADER . Market Information SUBHEADER

In [ ]:
ner_res = res[0]['ner_chunk'] # Document 0

sections = []
for ann in ner_res:
  sections.append( (ann.begin, ann.end) )
sections

Out[78]: [(0, 5),
 (7, 22),
 (25, 40),
 (144, 150),
 (153, 164),
 (240, 273),
 (323, 328),
 (331, 340),
 (581, 587),
 (589, 605),
 (659, 689),
 (692, 705),
 (708, 714),
 (716, 831),
 (834, 851)]

In [ ]:
section_texts = []
last_section = 0
last_section_name = ""
for s in sections:
  t = last_section_name + example2[last_section:s[0]]
  if t != '':
    section_texts.append(t)
  last_section = s[1]+1
  last_section_name = example2[s[0]:s[1]+1]
section_texts.append(last_section_name + example2[last_section:s[0]])


In [ ]:
OKGREEN = '\033[92m'
ENDC = '\033[0m'

In [ ]:
for t in section_texts:
  print(f"{OKGREEN}SECTION:{ENDC}\n{t}")

SECTION: 
PART I

 SECTION: 
ITEM 1. BUSINESS.

 SECTION: 
Company Overview
Inuvo is a technology company that develops and sells information technology solutions for marketing.

 SECTION: 
ITEM 1A. 
 SECTION: 
RISK FACTORS.
An investment in our common stock involves a significant degree of risk.

 SECTION: 
ITEM 1B. UNRESOLVED STAFF COMMENTS.
Not applicable to a smaller reporting company.

 SECTION: 
ITEM 2. 
 SECTION: 
PROPERTIES.
Our corporate headquarters are located in Little Rock, Arkansas where we entered into a five-year agreement to lease office space on October 1, 2015 and amended the lease as of February 1, 2021
Angeles, CA, San Jose, CA and Secaucus, NJ.

 SECTION: 
ITEM 3. 
 SECTION: 
LEGAL PROCEEDINGS.
We are not party to any pending legal proceedings.

 SECTION: 
ITEM 4. MINE SAFETY DISCLOSURES.

 SECTION: 
Not applicable.

 SECTION: 
PART II

 SECTION: 
ITEM 5. MARKET FOR REGISTRANT’S COMMON EQUITY, RELATED STOCKHOLDER MATTERS AND ISSUER PURCHASES OF EQUITY SECURITIES.

 SECTION: 
Market Information

... or automatically using ChunkSentenceSpliting...

In [ ]:
chunkSentenceSplitter = finance.ChunkSentenceSplitter()\
    .setInputCols("document","ner_chunk")\
    .setOutputCol("paragraphs")\
    .setGroupBySentences(False)

    
nlp_pipeline = nlp.Pipeline(stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        embeddings,
        ner_model,
        ner_converter,
        chunkSentenceSplitter])


paragraphs = nlp_pipeline.fit(sdf).transform(sdf)

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
result_df = paragraphs.selectExpr("explode(paragraphs) as result").selectExpr("result.result","result.metadata.entity").toPandas()
result_df.head()

Out[83]:

,result,entity
0,PART I\n,HEADER
1,ITEM 1. BUSINESS.\n,HEADER
2,Company Overview\nInuvo is a technology company that develops and sells information technology solutions for marketing.\n,SUBHEADER
3,ITEM 1A.,SUBHEADER
4,RISK FACTORS.\nAn investment in our common stock involves a significant degree of risk.\n,SUBHEADER


#✔ 1.5. Paragraph Classification and the consequences of splitting

![image.png](/files/FINLEG/pc1.png)

In Finance NLP we have several Text Classifiers to detect some useful sections of a 10K (or 10Q) filing. For example:
```
- finclf_acquisitions_item
- finclf_business_item
- finclf_equity_item
- finclf_exhibits_item
- finclf_executives_item
- finclf_properties_item
- finclf_work_experience_item
- finclf_controls_procedures_item
- finclf_security_ownership_item
- finclf_executives_compensation_item
- finclf_financial_statements_item
- finclf_market_risk_item
- finclf_financial_conditions_item
- finclf_legal_proceedings_item
- finclf_risk_factors_item
```
All of these models are Binary Classifiers, which means they will return True or False, or better to say ,the name of the class (for example, `acquisitions`) if the class `acquisition` is found, or `other` if it's not an `acquisition`, but something else.

CLARIFICATION: We did not want to return `True` or `False` because these models can all be stuck one after another, so in case a clause triggers some classes at the same time (they may not be disjoint) we could be getting several `True`, which is less informative than retrieving directly the type of the classes detected `acquisitions` and `properties` for example.

##📌 Example of how splitting affects paragraph classification

In [ ]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

text_splitter = finance.TextSplitter() \
    .setInputCols(["document"]) \
    .setOutputCol("pages")\
    .setCustomBounds(["Table of Contents"])\
    .setUseCustomBoundsOnly(True)\
    .setExplodeSentences(True)

nlp_pipeline = nlp.Pipeline(stages=[
    document_assembler,
    text_splitter])

fit = nlp_pipeline.fit(spark.createDataFrame([[ cadence_sec10k ]]).toDF("text"))

In [ ]:
lp = nlp.LightPipeline(fit)

In [ ]:
res = lp.annotate(cadence_sec10k)
pages = res['pages']
pages = [p for p in pages if p.strip() != ''] # We remove empty pages

In [ ]:
candidates = [[pages[4]], [pages[84]], [pages[85]], [pages[86]], [pages[87]]]

In [ ]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

use_embeddings = nlp.UniversalSentenceEncoder.pretrained() \
    .setInputCols("document") \
    .setOutputCol("sentence_embeddings")

doc_classifier = finance.ClassifierDLModel.pretrained('finclf_work_experience_item', "en", "finance/models")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("category")

nlp_pipeline = nlp.Pipeline(stages=[
    document_assembler, 
    use_embeddings,
    doc_classifier])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
finclf_work_experience_item download started this may take some time.
[ | ][OK!]

In [ ]:
df = spark.createDataFrame(candidates).toDF("text")

model = nlp_pipeline.fit(df)

result = model.transform(df)
result.select('category.result').show()

+-----------------+
 result|
+-----------------+
 [other]|
 [other]|
 [other]|
[work_experience]|
 [other]|
+-----------------+

Page 86 has some information about people and their roles...

In [ ]:
print(pages[86][-800:])

ed Public Accounting Firm.
X
31.01
Certification of the Registrant’s Chief Executive Officer, Anirudh Devgan, pursuant to Rule 13a-14 of the Securities Exchange Act of 1934.
X
31.02
Certification of the Registrant’s Chief Financial Officer, John M. Wall, pursuant to Rule 13a-14 of the Securities Exchange Act of 1934.
X
32.01†
Certification of the Registrant’s Chief Executive Officer, Anirudh Devgan, pursuant to 18 U.S.C. Section 1350, as adopted pursuant to Section 906 of the Sarbanes-Oxley Act of 2002.
X
32.02†
Certification of the Registrant’s Chief Financial Officer, John M. Wall, pursuant to 18 U.S.C. Section 1350, as adopted pursuant to Section 906 of the Sarbanes-Oxley Act of 2002.
X
101.INS
Inline XBRL Instance Document.
X
101.SCH
Inline XBRL Taxonomy Extension Schema Document.
X
84

But page 4 too!

In [ ]:
print(pages[4][-800:])

of our customers.
Recent Acquisitions
During fiscal 2021, we continued to execute our Intelligent System Design strategy and expanded our product offerings and solutions into computational fluid dynamics (“CFD”) with our acquisitions of Belgium-based NUMECA International, a leader in CFD technology, and Pointwise, Inc, a leading provider of CFD meshing technology. The addition of these technologies and talent broadens our System Design and Analysis portfolio and expertise. 
Chief Executive Officer Transition
On December 15, 2021, Anirudh Devgan assumed the role of President and Chief Executive Officer of Cadence, replacing Lip-Bu Tan. Prior to his role as Chief Executive Officer, Dr. Devgan served as President of Cadence. Concurrently, Mr. Tan transitioned to the role of Executive Chair.
2

What happened?

In [ ]:
pages[4]

Out[92]: 'The IC and SoC design excellence requires technologies for custom IC, digital IC design and signoff, and functional verification, and leverages pre-built semiconductor IP. These tools, IP and associated services are specifically designed to meet the growing requirements of engineers designing increasingly complex chips across analog, digital and mixed-signal domains, and perform the associated verification tasks, including validation of low-level software running on the silicon model, thereby enabling design teams to manage complexity and verification throughput without commensurately increasing the team size or extending the project schedule, while reducing technical risks.\nThe second layer of our strategy centers around system innovation. It includes tools and services used for system design of the packages that encapsulate the ICs and the PCBs, system simulation which includes electromagnetic, electro-thermal and other multi-physics analysis necessary as part of optimizing the full system’s performance, radio frequency (“RF”) and microwave systems, and embedded software.\nThe third layer of our strategy addresses pervasive intelligence in new electronics. It starts with providing solutions and services to develop AI-enhanced systems and includes machine learning and deep learning capabilities being added to the Cadence\n®\n technology portfolio to make IP and tools more automated and to produce optimized results faster.\nOur software and emulation products also support cloud access to address the growing computational needs of our customers.\nRecent Acquisitions\nDuring fiscal 2021, we continued to execute our Intelligent System Design strategy and expanded our product offerings and solutions into computational fluid dynamics (“CFD”) with our acquisitions of Belgium-based NUMECA International, a leader in CFD technology, and Pointwise, Inc, a leading provider of CFD meshing technology. The addition of these technologies and talent broadens our System Design and Analysis portfolio and expertise. \nChief Executive Officer Transition\nOn December 15, 2021, Anirudh Devgan assumed the role of President and Chief Executive Officer of Cadence, replacing Lip-Bu Tan. Prior to his role as Chief Executive Officer, Dr. Devgan served as President of Cadence. Concurrently, Mr. Tan transitioned to the role of Executive Chair.\n2'

![image.png](/files/FINLEG/pc2.png)

Predicting at page level added too much information, and the part of the CEO got deluded by the rest of the information!

Also token size restrictions may have happened!

SOLUTION: A smaller lever of granularity

In [ ]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

text_splitter = finance.TextSplitter() \
    .setInputCols(["document"]) \
    .setOutputCol("paragraphs")

nlp_pipeline = nlp.Pipeline(stages=[
    document_assembler, 
    text_splitter])

In [ ]:
empty_data = spark.createDataFrame([[""]]).toDF("text")
text_splitting_pipe = nlp_pipeline.fit(empty_data)
text_splitting_lightpipe = nlp.LightPipeline(text_splitting_pipe)

In [ ]:
res = text_splitting_lightpipe.annotate(pages[4])
paragraphs = res['paragraphs']
paragraphs = [p for p in paragraphs if p.strip() != ''] # We remove empty pages

In [ ]:
candidates = [[x] for x in paragraphs]

In [ ]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

use_embeddings = nlp.UniversalSentenceEncoder.pretrained() \
    .setInputCols("document") \
    .setOutputCol("sentence_embeddings")

doc_classifier = finance.ClassifierDLModel.pretrained('finclf_work_experience_item', "en", "finance/models")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("category")

nlp_pipeline = nlp.Pipeline(stages=[
    document_assembler, 
    use_embeddings,
    doc_classifier])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
finclf_work_experience_item download started this may take some time.
[ | ][OK!]

In [ ]:
df = spark.createDataFrame(candidates).toDF("text")
model = nlp_pipeline.fit(df)
result = model.transform(df)
result.select('category.result').show()

+-----------------+
 result|
+-----------------+
 [other]|
 [other]|
 [other]|
 [other]|
 [other]|
 [other]|
 [other]|
 [other]|
 [other]|
[work_experience]|
[work_experience]|
[work_experience]|
 [other]|
+-----------------+